In [1]:
import numpy as np
import tensorflow
import gzip
import sys
import pickle
import keras
import math
import os
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, Lambda
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K

In [2]:

import numpy as np
import tensorflow
import keras
import glob, os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
class DataLoader:
    num_images_1 = 49 # number of images in type 1 sets (1-77)
    num_images_2 = 64 # number of images in type 2 sets (82-128)
    input_shape = []
    train_labels = []
    test_labels = []
    train_data = []
    test_data = []
    input_shape = []
    # accepts the name of a directory and the name of a .npy file as strings
    # loads data from the given .npy if it exists, otherwise loads data from
    # raw images and saves it to a .npy file for future runs
    # returns a numpy array representation of
    # the image set in the given directiory
    def __load_images(self, directories, img_rows, img_cols):
        image_set = []
        cwd = os.getcwd() # save current working directory
        for directory_num in directories:
            new_images = []

            os.chdir(r"C:\Users\sulai\VIRTUAL INTERNSHIP\Virtual internship\Cleaned 9 scans\scan%s" % directory_num) # switch to directory for image files
            os.chdir(r"D:\Cleaned\scan%s" % directory_num) # switch to directory for image files
            if os.path.isfile("path%s.npy" % directory_num):
                new_images = np.load("path%s.npy" % directory_num)
            else:
                for file in glob.glob("*max.png"): # only loads the 'max' image from each view
                    img = image.load_img(file, target_size=(img_rows, img_cols))
                    img_array = image.img_to_array(img)
                    #print("array", directory_num, img_array.shape)
                    # do some other preprocessing here?
                    new_images.append(img_array)
                np.save("path%s.npy" % directory_num, new_images);

            if not np.array(image_set).size:
                image_set = new_images
            else:
                image_set= np.concatenate((image_set, new_images), 0)
        os.chdir(cwd) # switch back to previous working directory
        image_set = np.array(image_set)
        #preprocess input
        if K.image_data_format() == 'channels_first':
            image_set = image_set.reshape(image_set.shape[0], 3, img_rows, img_cols)
            self.input_shape = (3, img_rows, img_cols)
        else:
            image_set = image_set.reshape(image_set.shape[0], img_rows, img_cols, 3)
            self.input_shape = (img_rows, img_cols, 3)
        image_set = image_set.astype('float32')
        image_set /= 255
        return image_set
    # accepts an array of categoryIDs as a parameter
    # loads ground relative pose data for images in those categories
    def __load_labels(self, categoryIDs):
        labels = []
        data_files = [r"data/train_data_mvs.txt", r"data/test_data_mvs.txt"]
        #data_files = [r"data/train_data_mvs.txt", r"data/test_data_mvs.txt"]
        for file in data_files:
            f = open(file)
            for line in f:
                if line[0].isdigit():
                    sl = line.split()
                    nextTuple = (int(sl[0]), int(sl[1]), int(sl[2]),
                                 float(sl[3]), float(sl[4]), float(sl[5]),
                                 float(sl[6]), float(sl[7]), float(sl[8]), float(sl[9]))
                    if nextTuple[2] in categoryIDs:
                        labels.append(nextTuple);
            f.close()
        return np.array(labels)
    # accepts array of labels with image identifiers
    # returns shuffled labels split into training and testing sets
    def __organize_labels(self, labels):
        np.random.shuffle(labels)
        num_labels = np.array(labels).shape[0]
        train_index = int(0.8*num_labels)
        train_labels = labels[:train_index,:]
        test_labels = labels[train_index:,:]
        np.savetxt('index_spp.txt', train_labels, delimiter=' ')
        return train_labels, test_labels

    # accepts arrays of training and testing labels, 
    # array of images, and array of category IDs
    # returns arrays of image tuples representing the training and
    # testing datasets
    def __organize_data(self, train_labels, test_labels, images, categoryIDs):
        train_data = []
        test_data = []
        for label in train_labels:
            if label[2] <= 77: # currently can't category IDs above 77
                mult = int(categoryIDs.index(label[2]))
                img1_index = (mult * self.num_images_1) + int(label[0]) - 1
                img2_index = (mult * self.num_images_1) + int(label[1]) - 1
                image_tuple = (images[img1_index], images[img2_index])
                train_data.append(image_tuple)
        for label in test_labels:
            if label[2] <= 77: # currently can't category IDs above 77
                mult = int(categoryIDs.index(label[2]))
                img1_index = (mult * self.num_images_1) + int(label[0]) - 1
                img2_index = (mult * self.num_images_1) + int(label[1]) - 1
                image_tuple = (images[img1_index], images[img2_index])
                test_data.append(image_tuple)
        return np.array(train_data), np.array(test_data)
    # accepts arrays of training and testing labels
    # returs same arrays with image identifying data removed
    # final arrays have form: [relative translation, relative orientation]
    #						  [[x, y, z] [q1, q2, q3, q4]]
    def __clean_labels(self, train_labels, test_labels):
        return train_labels[:,3:], test_labels[:,3:]
    # returns shuffled training and test data consisting of
    # lists of image pairs with indicies [pair_num, image_num, width, height, depth]
    def get_data(self):
        return self.train_data, self.test_data
    # returns shuffled training and test labels with form:
    # [x, y, z, q1, q2, q3, q4]
    def get_labels(self):
        return self.train_labels, self.test_labels
    def get_input_shape(self):
        return self.input_shape
    def __init__(self, categoryIDs, img_rows, img_cols):
        images = self.__load_images(categoryIDs, img_rows, img_cols)
        labels = self.__load_labels(categoryIDs)
        self.train_labels, self.test_labels = self.__organize_labels(labels)
        self.train_data, self.test_data = self.__organize_data(self.train_labels, self.test_labels, images, categoryIDs)
        self.train_labels, self.test_labels = self.__clean_labels(self.train_labels, self.test_labels)



beta = 10
epochs = 10
def custom_objective(y_true, y_pred):
    error = K.square(y_pred - y_true)
    transMag = K.sqrt(error[0] + error[1] + error[2])
    orientMag = K.sqrt(error[3] + error[4] + error[5] + error[6])
    return K.mean(transMag + (beta * orientMag))
def dot_product(v1, v2):
    return sum((a*b) for a,b in zip(v1,v2))
def length(v):
    return math.sqrt(dot_product(v,v))
def compute_mean_error(y_true, y_pred):
    trans_error = 0
    orient_error = 0
    for i in range(0,y_true.shape[0]):
        trans_error += math.acos(dot_product(y_true[i,:3], y_pred[i,:3])/
                                 (length(y_true[i,:3]) * length(y_pred[i,:3])))
        orient_error += math.acos(dot_product(y_true[i,3:], y_pred[i,3:])/
                                  (length(y_true[i,3:]) * length(y_pred[i,3:])))
    mean_trans = trans_error / y_true.shape[0]
    mean_orient = orient_error / y_true.shape[0]
    return mean_trans, mean_orient
def create_conv_branch(input_shape):
    model = Sequential()
    model.add(Conv2D(96, kernel_size=(11,11),
                     strides=4, padding='valid',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(256, kernel_size=(5,5),
                     strides=1, padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=1))
    model.add(Conv2D(384, kernel_size=(3,3),
                     strides=1, padding='same',
                     activation='relu'))
    model.add(Conv2D(384, kernel_size=(3,3),
                     strides=1, padding='same',
                     activation='relu'))
    model.add(Conv2D(256, kernel_size=(3,3),
                     strides=1, padding='same',
                     activation='relu'))
    # replace with SPP if possible
    model.add(MaxPooling2D(pool_size=(3,3), strides=2))
    return model
if __name__ == "__main__":

    img_rows, img_cols = 227, 227
    category_IDs = [6] # category IDs from which to pull test and training data
    model_name = 'huge_model_10epoch.h5'
    model = None
    # load training and testing data:
    loader = DataLoader(category_IDs, img_rows, img_cols)
    train_data, test_data = loader.get_data()
    train_labels, test_labels = loader.get_labels()
    input_shape = loader.get_input_shape()
    # define structure of convolutional branches
    conv_branch = create_conv_branch(input_shape)
    branch_a = Input(shape=input_shape)
    branch_b = Input(shape=input_shape)
    processed_a = conv_branch(branch_a)
    processed_b = conv_branch(branch_b)
    # compute distance between outputs of the CNN branches
    # not sure if euclidean distance is right here
    # merging or concatenating inputs may be more accurate
    #distance = Lambda(euclidean_distance, 
    #				  output_shape = eucl_dist_output_shape)([processed_a, processed_b])
    regression = keras.layers.concatenate([processed_a, processed_b])
    regression = Flatten()(regression) # may not be necessary
    output = Dense(7, kernel_initializer='normal', name='output')(regression)
    model = Model(inputs=[branch_a, branch_b], outputs=[output])
    model.compile(loss=custom_objective, 
                  optimizer=keras.optimizers.Adam(lr=.0001, decay=.00001),
                  metrics=['accuracy'])
    if os.path.isfile(model_name):
        print("model", model_name, "found")
        model.load_weights(model_name)
        print("model loaded from file")
    else:

        model.fit([train_data[:,0], train_data[:,1]], train_labels,
                  batch_size=128,
                  epochs = epochs,
                  validation_split=0.1,
                  shuffle=True)
        model.save_weights(model_name)
        print("model saved as file", model_name)
    #pred = model.predict([train_data[:,0], train_data[:,1]])
    #train_trans, train_orient = compute_mean_error(pred, train_labels)
    pred = model.predict([test_data[:,0], test_data[:,1]])
    test_trans, test_orient = compute_mean_error(pred, test_labels)
    np.savetxt('pred.txt', pred, delimiter=' ')
    np.savetxt('labels.txt', test_labels, delimiter=' ')
    #print('* Mean translation error on training set: %0.2f' % (train_trans))
    #print('* Mean orientation error on training set: %0.2f' % (train_orient))
    print('*     Mean translation error on test set: %0.2f' % (test_trans))
    print('*     Mean orientation error on test set: %0.2f' % (test_orient))

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_data_mvs.txt'